# PhotoFiTT: Cell activity analysis

The data is assumed to be organised in folders as follows. The Subcategories are the ones that the code uses to display the data. The labels will be identical to the folder name, so the same conditions should always have the same folder name.

   ```   
   -Raw-images (folder)
    |
    |--Biological-replica-date-1 (folder) [Subcaegory-00]
        |
        |--Cell density / UV Ligth / WL 475 light [Subcategory-01] 
           |
           |-- control-condition (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition1 (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition2 (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
        |
        |--Cell density / UV Ligth / WL 475 light [Subcategory-01]
        ...
   ```

## 1. Import required dependencies

In [1]:
import os
import numpy as np
import pandas as pd
import photofitt
from photofitt import display
from photofitt.analysis import extract_activity
from photofitt.utils import power_conversion, numerical_dose

## 2. Set the paths to the data and the output folders
- `path2images`: The path where the Raw-images are saved. For example "/home/ocb/HardDrive_4TB/PHX/SYNCHRO/Raw-images"
- `path2results`: The path where the notebook will save all the results. Once the cell activity is calculated, a CSV file together with the plots will be stored there.
- `frame_rate`: The time frame in the videos given in minutes (e.g., `frame_rate = 4`)


In [ ]:
path2images = 
path2results = 
frame_rate = 4 

## Prepare the directory to save the results
os.makedirs(output_path,exist_ok=True)
os.makedirs(os.path.join(output_path, "plots"), exist_ok=True)


## 3. Calculate cell activity.
This process can take long hours depending on the number of videos to analyse. If it is already calculated, do not run this cell again and jump to the next step

In [ ]:
images_path = "/home/ocb/HardDrive_4TB/EGM/PHX/DATA/DOWNSAMPLE/CHO-SYNCHRO/scaled_1.5709_results"
frame_rate = 4 # The time gap we will use to compute all the metrics
output_path =  "/home/ocb/HardDrive_4TB/EGM/PHX/ANALYSIS/SYNCHRO/NEW-ACTIVITY"
## Parameters for activity method estimation
method="intensity"

activity_metrics = extract_activity(path2images,
                                    method="intensity",
                                    save_steps=False,
                                    enhance_contrast=True,
                                    output_path=output_path,
                                    condition=None)

## Save the results
activity_metrics.to_csv(os.path.join(output_path, "data_activity_intensity.csv"))

## 4. Analyse cell activity


### 4.1 Inspect the cell activity accross time

In [ ]:
activity_metrics = pd.read_csv(os.path.join(output_path, "data_activity_intensity.csv"))
# If the data was organised as indicated, Subcategory-01 corresponds to the wavelength (e.g., 385nm illumination)
wavelength = np.unique(activity_metrics["Subcategory-01"])


hue="Light dose cat" # It's the variable that corresponds to the light dose (e.g., 60 J/cm2)
hue_order = ['0 J/cm2','0.6 J/cm2', '6 J/cm2', '60 J/cm2'] # Establish your own order if needed.
ylim = [0.0001, 0.0035] # This is the y-axis limit to plot everything similarly. Addapt it as needed. 
format_extension = 'pdf' # Format to save the plots.
for w in wavelength:
    activity_w = activity_metrics[activity_metrics["Subcategory-01"]==w].reset_index(drop=True)    

    # Plot the temporal distribution of the activity
    y = "mean activity" # Variable to display
    display.conditions_with_aggregates(activity_w, y,
                               f"Cell activity variance {w}",
                               os.path.join(output_path, "plots"),
                               f"activity_variance_{w}.png",
                               hue=hue, 
                               hue_order=hue_order,
                               ylim = ylim)
    
    ## Plot additive activity
    activity_w_dataframe, activity_w = cumulative_activity(activity_w, y, use_starting_point="None", starting_point=0)
    
    y_var = f"Cumulative {y}"
    name = "cell_cumulative_activity_{0}.{1}".format(w, format_extension)
    title = f"{w}"
    display.conditions(activity_w, y_var, 
               title, 
               hue, 
               os.path.join(output_path, "plots"), name, 
               hue_order=hue_order,
               palette=sns.color_palette("CMRmap_r", len(hue_order)) # change the palette if desired
               figsize=(4, 5)) # Change the aspect ratio of the figure if desired 
    